In [ ]:
!pip install bs4

In [ ]:
import requests
import pandas as pd
import time
import json
import logging
from bs4 import BeautifulSoup
import re
from requests.exceptions import RequestException

In [ ]:
# Настраиваем логирование
logging.basicConfig(
    filename='sephora_parse1.log',
    level=logging.INFO,
    format='api_parsing | %(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True
)
logging.info("Тестовая запись в лог")
# Список токенов
TOKENS = [
    "8bc3bea522msh941d5f19c472aa2p1d9362jsna422162a847e",
    "71df97361cmshfa8e56c9d1d52c9p16f4cbjsnd8371aa26010",
    "06cc033d84mshb5f1fe6ebc4cc23p1e5e17jsnfba965f47afe",
    "b1f98658a4mshceede2464f383d5p1db43ajsn01003e021291",
    "ee71f8cf5amsh9606a89731714bcp16e318jsnf25c70e56aae",
    "0ccc6b07f9msh9d97bb824c2932ep1c8e14jsn405f51cc35f1"
]
CURRENT_TOKEN_INDEX = 0  # Индекс текущего токена

In [ ]:

def get_headers():
    """Формируем заголовки запроса с текущим токеном."""
    return {
        "x-rapidapi-key": TOKENS[CURRENT_TOKEN_INDEX],
        "x-rapidapi-host": "sephora14.p.rapidapi.com"
    }

def get_brands():
    """
    Получаем список брендов.
    Если получаем 429 — переключаем токен и повторяем.
    Если всё исчерпано — возвращаем None.
    """
    global CURRENT_TOKEN_INDEX
    url = "https://sephora14.p.rapidapi.com/brands"

    while CURRENT_TOKEN_INDEX < len(TOKENS):
        try:
            response = requests.get(url, headers=get_headers())
            if response.status_code == 200:
                logging.info("Успешно получен список брендов.")
                return response.json()

            elif response.status_code == 429:
                logging.warning(f"Получен 429 при получении брендов. Токен {TOKENS[CURRENT_TOKEN_INDEX]} исчерпан. Переключаемся...")
                CURRENT_TOKEN_INDEX += 1
                if CURRENT_TOKEN_INDEX >= len(TOKENS):
                    logging.error("Все токены исчерпаны, бренды получить невозможно.")
                    return None
                # Пробуем заново с новым токеном
                continue

            else:
                logging.error(f"Неожиданный статус ответа: {response.status_code} при получении брендов.")
                return None

        except requests.exceptions.RequestException as e:
            logging.error(f"Ошибка при запросе списка брендов: {str(e)}")
            return None

    # Если вышли из цикла, значит токены закончились
    logging.error("Все токены исчерпаны — список брендов не получен.")
    return None

def process_products(products_data):
    """Преобразуем список продуктов в DataFrame с нужными полями."""
    if not products_data:
        return None

    df = pd.DataFrame(products_data)

    # Пример: извлекаем некоторые поля из 'currentSku'
    if 'currentSku' in df.columns:
        df['price'] = df['currentSku'].apply(lambda x: x.get('listPrice') if isinstance(x, dict) else None)
        df['isLimitedEdition'] = df['currentSku'].apply(lambda x: x.get('isLimitedEdition') if x else None)
        df['isLimitedTimeOffer'] = df['currentSku'].apply(lambda x: x.get('isLimitedTimeOffer') if x else None)
        df['skuType'] = df['currentSku'].apply(lambda x: x.get('skuType') if x else None)
        df['isAppExclusive'] = df['currentSku'].apply(lambda x: x.get('isAppExclusive') if x else None)
        df['isBI'] = df['currentSku'].apply(lambda x: x.get('isBI') if x else None)
        df['isBest'] = df['currentSku'].apply(lambda x: x.get('isBest') if x else None)
        df['isNatural'] = df['currentSku'].apply(lambda x: x.get('isNatural') if x else None)
        df['isNew'] = df['currentSku'].apply(lambda x: x.get('isNew') if x else None)
        df['isOnlineOnly'] = df['currentSku'].apply(lambda x: x.get('isOnlineOnly') if x else None)
        df['biExclusiveLevel'] = df['currentSku'].apply(lambda x: x.get('biExclusiveLevel') if x else None)
    return df

def get_brand_products(brand_id, page=1):
    """
    Запрос к /searchByBrand по конкретному brand_id и странице page.
    Если получаем 429 — переключаем токен и повторяем для той же позиции.
    Возвращаем кортеж (список_продуктов, total_pages).
    Если не вышло — (None, 0).
    """
    global CURRENT_TOKEN_INDEX
    url = "https://sephora14.p.rapidapi.com/searchByBrand"
    querystring = {
        "brandID": brand_id,
        "page": str(page),
        "sortBy": "NEW"
    }

    while CURRENT_TOKEN_INDEX < len(TOKENS):
        try:
            response = requests.get(url, headers=get_headers(), params=querystring)

            if response.status_code == 200:
                data = response.json()
                products = data.get('products', [])
                total_pages = data.get('totalPages', 1)
                return products, total_pages

            elif response.status_code == 429:
                logging.warning(f"429 при запросе бренда {brand_id}, стр. {page}. "
                                f"Токен {TOKENS[CURRENT_TOKEN_INDEX]} исчерпан. Переключаемся...")
                CURRENT_TOKEN_INDEX += 1
                if CURRENT_TOKEN_INDEX >= len(TOKENS):
                    logging.error("Все токены исчерпаны, дальнейшая работа невозможна.")
                    return None, 0
                # Повторим запрос на той же странице с новым токеном
                continue

            elif response.status_code == 500:
                logging.error(f"Серверная ошибка 500 для бренда {brand_id}, стр. {page}. Пропускаем бренд.")
                return None, 0

            else:
                logging.error(f"Неожиданный статус {response.status_code} для бренда {brand_id}, стр. {page}.")
                return None, 0

        except requests.exceptions.RequestException as e:
            logging.error(f"Исключение при запросе бренда {brand_id}, стр. {page}: {str(e)}")
            return None, 0

    # Если вышли, значит токены закончились
    logging.error("Все токены исчерпаны, запрос невозможен.")
    return None, 0

def main():
    logging.info("Запуск скрипта Sephora.")
    print('ok1')
    # 1. Получаем список брендов
    brands = get_brands()
    if not brands:
        logging.error("Список брендов не получен — завершаем работу.")
        return
    print('ok2')

    df_global = pd.DataFrame()

    # 2. Перебираем бренды подряд
    for brand_id in brands:
        logging.info(f"Обработка бренда: {brand_id}")

        # Запрашиваем первую страницу
        page = 1
        products, total_pages = get_brand_products(brand_id, page)
        if products is None:
            logging.warning(f"Пропускаем бренд {brand_id}, нет данных или ошибка.")
            continue

        # Обрабатываем продукты 1-й страницы
        temp_df = process_products(products)
        if temp_df is not None:
            temp_df['brand_id'] = brand_id
            df_global = pd.concat([df_global, temp_df], ignore_index=True)

        # Если страниц несколько, идём по остальным
        for page_num in range(2, total_pages + 1):
            products, _ = get_brand_products(brand_id, page_num)
            if products:
                temp_df = process_products(products)
                if temp_df is not None:
                    temp_df['brand_id'] = brand_id
                    df_global = pd.concat([df_global, temp_df], ignore_index=True)
            # Между запросами — небольшая пауза
            time.sleep(1)

        # Промежуточное сохранение после каждого бренда (по желанию)
        if not df_global.empty:
            df_global.to_csv('sephora_products_intermediate.csv', index=False)
            logging.info(f"Промежуточно сохранено {len(df_global)} строк.")
            print('ok')

        # Пауза перед следующим брендом
        time.sleep(2)

    # Итоговое сохранение
    if not df_global.empty:
        df_global.to_csv('sephora_products_final.csv', index=False)
        logging.info(f"Всего собрано {len(df_global)} продуктов. Итоговый файл: sephora_products_final.csv")
        print('ok4')
    else:
        logging.info("Продукты не были собраны, файл не создан.")
        print('not ok')

    logging.info("Скрипт завершён.")

if __name__ == "__main__":
    main()


ok1
ok2
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok4


In [ ]:
print(f"Всего строк после объединения: {len(df_global)}")

NameError: name 'df_global' is not defined

In [ ]:
!cat sephora_parse.log

cat: sephora_parse.log: No such file or directory


In [ ]:
def process_products(products_data):
    if not products_data:
        return None

    df = pd.DataFrame(products_data)
    df['price'] = df['currentSku'].apply(lambda x: x.get('listPrice') if x else None)
    df['isLimitedEdition'] = df['currentSku'].apply(lambda x: x.get('isLimitedEdition') if x else None)
    df['isLimitedTimeOffer'] = df['currentSku'].apply(lambda x: x.get('isLimitedTimeOffer') if x else None)
    df['skuType'] = df['currentSku'].apply(lambda x: x.get('skuType') if x else None)
    df['isAppExclusive'] = df['currentSku'].apply(lambda x: x.get('isAppExclusive') if x else None)
    df['isBI'] = df['currentSku'].apply(lambda x: x.get('isBI') if x else None)
    df['isBest'] = df['currentSku'].apply(lambda x: x.get('isBest') if x else None)
    df['isNatural'] = df['currentSku'].apply(lambda x: x.get('isNatural') if x else None)
    df['isNew'] = df['currentSku'].apply(lambda x: x.get('isNew') if x else None)
    df['isOnlineOnly'] = df['currentSku'].apply(lambda x: x.get('isOnlineOnly') if x else None)
    df['biExclusiveLevel'] = df['currentSku'].apply(lambda x: x.get('biExclusiveLevel') if x else None)
    df = df.drop(columns=[col for col in ['heroImage', 'altImage', 'currentSku', 'productId', 'onSaleData'] if col in df.columns])
    return df

In [ ]:
def get_brand_products(brand_id, page=1, max_retries=3):
    url = "https://sephora14.p.rapidapi.com/searchByBrand"

    querystring = {
        "brandID": brand_id,
        "page": str(page),
        "sortBy": "NEW"
    }

    headers = {
        "x-rapidapi-key": "057acdba41msh5565e18a973aa7ap138573jsn094c698cf753",
        "x-rapidapi-host": "sephora14.p.rapidapi.com"
    }

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=querystring)

            if response.status_code == 200:
                data = response.json()
                if 'products' in data:
                    total_pages = data.get('totalPages', 1)
                    return data['products'], total_pages
                else:
                    print(f"Нет данных о продуктах для бренда {brand_id}")
                    return None, 0

            elif response.status_code == 429:
                wait_time = 10 * (attempt + 1)  # Увеличиваем время ожидания с каждой попыткой
                print(f"Превышен лимит запросов. Ожидание {wait_time} секунд...")
                time.sleep(wait_time)
                continue  # Пробуем снова после ожидания

            elif response.status_code == 500:
                print(f"Ошибка сервера для бренда {brand_id}. Пропускаем бренд.")
                return None, 0

            else:
                print(f"Неожиданный статус ответа: {response.status_code}")
                if attempt < max_retries - 1:
                    time.sleep(10)
                    continue
                return None, 0

        except Exception as e:
            print(f"Исключение при запросе: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(10)
                continue
            return None, 0

    return None, 0

In [ ]:
df = pd.DataFrame()
brands = get_brands()

if brands:
    start_index = brands.index('topicals') if 'topicals' in brands else 0
    brands = brands[start_index:]
    print(f"Начинаем сбор данных с бренда: {brands[0]}")
    for brand_id in brands:
        print(f"Обработка бренда: {brand_id}")
        page = 1

        products, total_pages = get_brand_products(brand_id, page)

        if products is None:
            print(f"Пропускаем бренд {brand_id} и переходим к следующему")
            time.sleep(5)
            continue

        temp_df = process_products(products)
        if temp_df is not None:
            temp_df['brand_id'] = brand_id
            df = pd.concat([df, temp_df], ignore_index=True)

        for page in range(2, total_pages + 1):
            print(f"Обработка страницы {page} из {total_pages} для бренда {brand_id}")
            products, _ = get_brand_products(brand_id, page)
            if products:
                temp_df = process_products(products)
                if temp_df is not None:
                    temp_df['brand_id'] = brand_id
                    df = pd.concat([df, temp_df], ignore_index=True)
            time.sleep(1)
        if len(df) > 0:
            df.to_csv('sephora_products_intermediate.csv', index=False)
            print(f"Сохранено {len(df)} продуктов в промежуточный файл")
        time.sleep(2)

    print(f"Всего собрано продуктов: {len(df)}")
    df.to_csv('sephora_products_final.csv', index=False)
else:
    print("Не удалось получить список брендов")

Начинаем сбор данных с бренда: topicals
Обработка бренда: topicals
Сохранено 11 продуктов в промежуточный файл
Обработка бренда: touchland
Сохранено 21 продуктов в промежуточный файл
Обработка бренда: tower-28
Сохранено 41 продуктов в промежуточный файл
Обработка бренда: tula-skincare
Сохранено 73 продуктов в промежуточный файл
Обработка бренда: tweezerman
Сохранено 80 продуктов в промежуточный файл
Обработка бренда: urban-decay
Сохранено 136 продуктов в промежуточный файл
Обработка бренда: valentino
Сохранено 171 продуктов в промежуточный файл
Обработка бренда: vegamour
Сохранено 205 продуктов в промежуточный файл
Обработка бренда: velour-lashes
Сохранено 216 продуктов в промежуточный файл
Обработка бренда: verb-hair-care
Сохранено 253 продуктов в промежуточный файл
Обработка бренда: versace
Сохранено 294 продуктов в промежуточный файл
Обработка бренда: viktor-rolf
Сохранено 317 продуктов в промежуточный файл
Обработка бренда: violet-voss
Ошибка сервера для бренда violet-voss. Пропуск

In [ ]:
df0 = pd.read_csv('sephora_products_intermediate.csv')
df1 = pd.read_csv('sephora_products_intermediate (1).csv')
df2 = pd.read_csv('sephora_products_intermediate (2).csv')
df3 = pd.read_csv('sephora_products_intermediate (3).csv')
df4 = pd.read_csv('sephora_products_intermediate (4).csv')
df5 = pd.read_csv('sephora_products_intermediate (5).csv')

In [ ]:
df_sephora = pd.concat([df0, df1, df2, df3, df4, df5], ignore_index=True)
print(f"Всего строк после объединения: {len(df_sephora)}")

Всего строк после объединения: 6843


In [ ]:
df.head()

,brandName,displayName,rating,reviews,targetUrl,price,isLimitedEdition,isLimitedTimeOffer,skuType,isAppExclusive,isBI,isBest,isNatural,isNew,isOnlineOnly,biExclusiveLevel,brand_id,moreColors
0,ABBOTT,Papaya Isla Eau de Parfum,4.5909,22,https://www.sephora.com/product/papaya-isla-ea...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
1,ABBOTT,Crescent Beach Eau de Parfum,4.5652,23,https://www.sephora.com/product/abbott-crescen...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
2,ABBOTT,Seqouia Eau de Parfum,4.1852,27,https://www.sephora.com/product/abbott-sequoia...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
3,ABBOTT,The Cape Eau de Parfum,4.5789,19,https://www.sephora.com/product/abbott-the-cap...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,True,none,abbott,NaN
4,ABBOTT,Big Sky Eau de Parfum,4.7500,12,https://www.sephora.com/product/abbott-big-sky...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN


ЧАСТЬ СО СКРАППИНГОМ

In [ ]:
%pip install -q google-colab-selenium

In [ ]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options

options = Options()

options.add_argument("--window-size=1920,1080")  # устанавливаем размер окна
options.add_argument("--disable-infobars")  # отключаем информационную панель
options.add_argument("--disable-popup-blocking")  # отключаем всплывающие окна
options.add_argument("--ignore-certificate-errors")  # игнорируем ошибки сертификатов
options.add_argument("--incognito")  # используем браузер в режиме инкогнито


driver = gs.Chrome(options=options)

driver.get('https://rivegauche.ru/brands')

In [ ]:
import logging

logging.basicConfig(
    filename='rive_gauche_parse.log',
    level=logging.INFO,
    format='scraping | %(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True
)
logging.info("Тестовая запись в лог")

In [ ]:
from selenium.webdriver.common.by import By

brands = driver.find_elements(By.XPATH, '//a[contains(@href, "/brands/")]') # ищем ссылки, содержащие "/brands/"

In [ ]:
brand_links = []

In [ ]:
for brand in brands:
    href = brand.get_attribute('href') # берем ссылку на бренд
    name = brand.get_attribute('text') # берем название бренда
    if href and name:
        brand_links.append([name, href])

In [ ]:
brand_links

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [ ]:
products_data = []

In [ ]:
driver = gs.Chrome(options=options)

for brand in brand_links:
    brand_name, brand_link = brand
    driver.get(brand_link)

    # Ищем ссылки, содержащие "/product/"
    try:
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/product/")]'))
        )
    # если ссылка не найдена в течение 1 секунды
    except:
        logging.info(f"Продукты не найдены для {brand_name}. Продолжаем...")
        continue

    # чтобы избежать дубликатов используем множество
    product_links_set = set()

    # скролл страницы, пока товары появляются
    last_count = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # скролл
        time.sleep(1)

        product_links = driver.find_elements(By.XPATH, '//a[contains(@href, "/product/")]')

        for product in product_links:
            product_href = product.get_attribute("href")
            if product_href:
                product_links_set.add(product_href)

        if len(product_links_set) == last_count:
            break
        last_count = len(product_links_set)

    for product_href in product_links_set:
        products_data.append([brand_name, product_href])

    logging.info(f"Ссылки на продукты добавлены для {brand_name}")

driver.quit()

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import re
from requests.exceptions import RequestException

In [ ]:
detailed_products_data = []

In [ ]:
def process_product_page(product_href):
    try:
        response = requests.get(product_href, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        description_meta = soup.find('meta', {'itemprop': 'description'})
        description = description_meta.get('content') if description_meta else None
        table = soup.find('table')
        if table:
            rows = table.find_all('tr')
        else:
            rows = []
        size = None
        category = None
        production = None
        result = None

        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 2:
                label = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)

                if label in ['Объем, мл', 'Вес, г']:
                    size = f"{value} {'мл' if label == 'Объем, мл' else 'г'}"
                elif label == 'Категория':
                    category = value
                elif label == 'Производство':
                    production = value
                elif label == 'Результат':
                    result = value

        return description, size, category, production, result

    except Exception as e:
        logging.warning(f"Ошибка при обработке страницы продукта {product_href}: {str(e)}")
        return None, None, None, None, None

In [ ]:
driver = gs.Chrome(options=options)

for product in products_data:
    brand_name, product_href = product
    driver.get(product_href)

    # Ждем загрузки товара
    try:
        WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.TAG_NAME, "h1"))
        )
    except:
        logging.info(f"Товар не загрузился: {product_href}. Продолжаем...")
        continue

    # 1. название товара
    try:
        product_name = driver.find_element(By.TAG_NAME, "h1").text.strip()
    except:
        product_name = None

    # 2. рейтинг
    try:
        rating = driver.find_element(By.XPATH, '//div[@class="rating"]/span').text.strip()
    except:
        rating = None

    # 3. количество отзывов
    try:
        reviews_count = driver.find_element(By.XPATH, '//div[@class="reviews-count"]').text.strip()
        reviews_count = int(re.sub(r'\D', '', reviews_count))
    except:
        reviews_count = 0

    # 4. хит/не хит
    try:
        is_hit = True if driver.find_element(By.XPATH, '//div[contains(text(), " Хит ")]') else False
    except:
        is_hit = False

    # 5. только в магазинах
    try:
        only_in_stores = True if driver.find_element(By.XPATH, '//div[contains(text(), " Только для самовывоза из магазина ")]') else False
    except:
        only_in_stores = False

    # 6. цена
    try:
        price = driver.find_element(By.XPATH, '//div/span[@class="base-value"]').text.strip()
        price = int(re.sub(r'\D', '', price))
    except:
        price = None

    # 7. описание, размер, категория, производство, результат
    description, size, category, production, result = process_product_page(product_href)


    detailed_products_data.append([
        brand_name, product_href, product_name, rating, reviews_count, price, is_hit, only_in_stores, description, size, category, production, result
    ])

    logging.info(f"Добавлен продукт {product_name}")

driver.quit()

In [ ]:
detailed_products_data

In [ ]:
df_products_data = pd.DataFrame(detailed_products_data,
                                columns=['brand_name', 'product_link', 'product_name',
                                         'rating', 'reviews_count', 'price', 'is_hit',
                                         'only_in_stores', 'description', 'size', 'category',
                                         'production', 'result'])

In [ ]:
df_products_data.to_csv('rive_gauche_products.csv', encoding='utf-8-sig', index=False)

NameError: name 'df_products_data' is not defined

# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import re
import numpy as np

In [ ]:
df_riv = pd.read_csv('rive_gauche_products.csv')
df_sephora = pd.read_csv('sephora_products_final (1).csv')

In [ ]:
df_riv.head()

,brand_name,product_link,product_name,rating,reviews_count,price,is_hit,only_in_stores,description,size,category,production,result
0,ACCA KAPPA,https://rivegauche.ru/product/acca-kappa-giall...,Acca Kappa Giallo Elicriso Intense Moisturizin...,NaN,0,3500.0,False,False,Интенсивно увлажняющий и бодрящий гель для ван...,500 мл,Уход за телом,Италия,Очищение
1,ACCA KAPPA,https://rivegauche.ru/product/acca-kappa-musch...,Acca Kappa Muschio Bianco White Moss After Sha...,NaN,0,3800.0,False,False,Питательная и успокаивающая эмульсия после бри...,125 мл,Средства для бритья,Италия,Увлажнение
2,ACCA KAPPA,https://rivegauche.ru/product/acca-kappa-sakur...,Acca Kappa Sakura Tokyo Eau de Parfum,NaN,0,2990.0,False,False,В Японии с конца марта и до начала апреля ежег...,15 мл,Парфюмерия,Италия,NaN
3,ACCA KAPPA,https://rivegauche.ru/product/acca-kappa-musch...,Acca Kappa Muschio Bianco Moisturizing & Invig...,NaN,0,5300.0,False,False,"Молочко для тела смягчает, интенсивно увлажняе...",300 мл,Уход за телом,Италия,Увлажнение
4,ACCA KAPPA,https://rivegauche.ru/product/acca-kappa-sakur...,Acca Kappa Sakura Tokyo Hand Cream,NaN,0,1750.0,False,False,Увлажняющий и защитный крем для рук Sakura обл...,75 мл,Уход за телом,Италия,Увлажнение


In [ ]:
pd.set_option('display.max_columns', None)
df_sephora.head()

,brandName,currentSku,displayName,heroImage,altImage,onSaleData,productId,rating,reviews,targetUrl,price,isLimitedEdition,isLimitedTimeOffer,skuType,isAppExclusive,isBI,isBest,isNatural,isNew,isOnlineOnly,biExclusiveLevel,brand_id,moreColors
0,ABBOTT,"{'biExclusiveLevel': 'none', 'imageAltText': '...",Papaya Isla Eau de Parfum,https://www.sephora.com/productimages/sku/s267...,https://www.sephora.com/productimages/product/...,NONE,P505624,4.5909,22,https://www.sephora.com/product/papaya-isla-ea...,$88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
1,ABBOTT,"{'biExclusiveLevel': 'none', 'imageAltText': '...",Crescent Beach Eau de Parfum,https://www.sephora.com/productimages/sku/s258...,https://www.sephora.com/productimages/product/...,NONE,P483079,4.5652,23,https://www.sephora.com/product/abbott-crescen...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
2,ABBOTT,"{'biExclusiveLevel': 'none', 'imageAltText': '...",Seqouia Eau de Parfum,https://www.sephora.com/productimages/sku/s258...,https://www.sephora.com/productimages/product/...,NONE,P483130,4.1852,27,https://www.sephora.com/product/abbott-sequoia...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
3,ABBOTT,"{'biExclusiveLevel': 'none', 'imageAltText': '...",Big Sky Eau de Parfum,https://www.sephora.com/productimages/sku/s258...,https://www.sephora.com/productimages/product/...,NONE,P483071,4.7500,12,https://www.sephora.com/product/abbott-big-sky...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,abbott,NaN
4,ABBOTT,"{'biExclusiveLevel': 'none', 'imageAltText': '...",The Cape Eau de Parfum,https://www.sephora.com/productimages/sku/s258...,https://www.sephora.com/productimages/product/...,NONE,P483139,4.5789,19,https://www.sephora.com/product/abbott-the-cap...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,True,none,abbott,NaN


In [ ]:
df_sephora = df_sephora.drop(columns=['currentSku', 'heroImage', 'altImage', 'productId', 'brand_id'])

In [ ]:
df_sephora.head()

,brandName,displayName,onSaleData,rating,reviews,targetUrl,price,isLimitedEdition,isLimitedTimeOffer,skuType,isAppExclusive,isBI,isBest,isNatural,isNew,isOnlineOnly,biExclusiveLevel,moreColors
0,ABBOTT,Papaya Isla Eau de Parfum,NONE,4.5909,22,https://www.sephora.com/product/papaya-isla-ea...,$88.00,False,False,Standard,False,False,False,False,False,False,none,NaN
1,ABBOTT,Crescent Beach Eau de Parfum,NONE,4.5652,23,https://www.sephora.com/product/abbott-crescen...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,NaN
2,ABBOTT,Seqouia Eau de Parfum,NONE,4.1852,27,https://www.sephora.com/product/abbott-sequoia...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,NaN
3,ABBOTT,Big Sky Eau de Parfum,NONE,4.7500,12,https://www.sephora.com/product/abbott-big-sky...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,False,none,NaN
4,ABBOTT,The Cape Eau de Parfum,NONE,4.5789,19,https://www.sephora.com/product/abbott-the-cap...,$31.00 - $88.00,False,False,Standard,False,False,False,False,False,True,none,NaN


In [ ]:
new_rows = []
for idx, row in df_sephora.iterrows():
    price_str = row['price']

    if '-' in price_str:
        prices = re.findall(r'\$(\d+\.\d+)', price_str)

        if len(prices) == 2:
            min_price = float(prices[0])
            max_price = float(prices[1])

            small_row = row.copy()
            small_row['price'] = min_price
            small_row['size'] = 'small'
            new_rows.append(small_row)

            big_row = row.copy()
            big_row['price'] = max_price
            big_row['size'] = 'big'
            new_rows.append(big_row)

        else:
            row['price'] = float(re.sub(r'[^\d.]', '', price_str))
            row['size'] = 'standard'
            new_rows.append(row)

    else:
        row['price'] = float(re.sub(r'[^\d.]', '', price_str))
        row['size'] = 'standard'
        new_rows.append(row)

df_sephora = pd.DataFrame(new_rows)

In [ ]:
df_sephora.head()

,brandName,displayName,onSaleData,rating,reviews,targetUrl,price,isLimitedEdition,isLimitedTimeOffer,skuType,isAppExclusive,isBI,isBest,isNatural,isNew,isOnlineOnly,biExclusiveLevel,moreColors,size
0,ABBOTT,Papaya Isla Eau de Parfum,NONE,4.5909,22,https://www.sephora.com/product/papaya-isla-ea...,88.0,False,False,Standard,False,False,False,False,False,False,none,NaN,standard
1,ABBOTT,Crescent Beach Eau de Parfum,NONE,4.5652,23,https://www.sephora.com/product/abbott-crescen...,31.0,False,False,Standard,False,False,False,False,False,False,none,NaN,small
1,ABBOTT,Crescent Beach Eau de Parfum,NONE,4.5652,23,https://www.sephora.com/product/abbott-crescen...,88.0,False,False,Standard,False,False,False,False,False,False,none,NaN,big
2,ABBOTT,Seqouia Eau de Parfum,NONE,4.1852,27,https://www.sephora.com/product/abbott-sequoia...,31.0,False,False,Standard,False,False,False,False,False,False,none,NaN,small
2,ABBOTT,Seqouia Eau de Parfum,NONE,4.1852,27,https://www.sephora.com/product/abbott-sequoia...,88.0,False,False,Standard,False,False,False,False,False,False,none,NaN,big


In [ ]:
products_count = {
    'Рив Гош': len(df_riv),
    'Sephora': len(df_sephora)
}

brands_count = {
    'Рив Гош': df_riv['brand_name'].nunique(),
    'Sephora': df_sephora['brandName'].nunique()
}

avg_products_per_brand = {
    'Рив Гош': len(df_riv) / df_riv['brand_name'].nunique(),
    'Sephora': len(df_sephora) / df_sephora['brandName'].nunique()
}

fig = make_subplots(rows=1, cols=3,
                   subplot_titles=('Количество товаров', 'Количество брендов', 'Среднее кол-во товаров на 1 бренд'))

fig.add_trace(
    go.Bar(
        x=list(products_count.keys()),
        y=list(products_count.values()),
        marker_color=['salmon', 'orange'],
        text=list(products_count.values()),
        textposition='auto'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=list(brands_count.keys()),
        y=list(brands_count.values()),
        marker_color=['salmon', 'orange'],
        text=list(brands_count.values()),
        textposition='auto'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=list(avg_products_per_brand.keys()),
        y=list(avg_products_per_brand.values()),
        marker_color=['salmon', 'orange'],
        text=[f"{val:.1f}" for val in avg_products_per_brand.values()],
        textposition='auto'
    ),
    row=1, col=3
)

fig.update_layout(
    title_text='Сравнение Рив Гош и Sephora',
    height=500,
    width=1200,
    showlegend=False
)

fig.update_yaxes(range=[0, max(products_count.values()) * 1.1], row=1, col=1)
fig.update_yaxes(range=[0, max(brands_count.values()) * 1.1], row=1, col=2)
fig.update_yaxes(range=[0, max(avg_products_per_brand.values()) * 1.1], row=1, col=3)

fig.show()

У Рив Гош более разнообразный ассортимент как товаров, так и брендов. Также в среднем на один бренд в Рив Гош представлено больше продуктов, чем в Sephora.

In [ ]:
top10_brands_sp = df_sephora['brandName'].value_counts().head(10)
top10_brands_rg = df_riv['brand_name'].value_counts().head(10)

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Рив Гош – Топ-10 брендов (по кол-ву товаров)",
                                   "Sephora – Топ-10 брендов (по кол-ву товаров)"),
                    horizontal_spacing=0.2)

fig.add_trace(
    go.Bar(
        x=top10_brands_rg.values,
        y=list(reversed(top10_brands_rg.index)),
        orientation='h',
        marker_color='#6a9955',
        text=top10_brands_rg.values,
        textposition='auto'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=top10_brands_sp.values,
        y=list(reversed(top10_brands_sp.index)),
        orientation='h',
        marker_color='#569cd6',
        text=top10_brands_sp.values,
        textposition='auto'
    ),
    row=1, col=2
)

fig.update_layout(
    height=500,
    width=1200,
    showlegend=False,
    title_text="Топ-10 брендов по количеству товаров"
)

fig.update_xaxes(title_text="Кол-во товаров", row=1, col=1)
fig.update_xaxes(title_text="Кол-во товаров", row=1, col=2)
fig.update_yaxes(title_text="Бренд", row=1, col=1)
fig.update_yaxes(title_text="Бренд", row=1, col=2, showticklabels=True)

fig.show()

В Рив Гош Топ-10 брендов включают как международные марки (например, URIAGE, CLARINS, PUPA), так и российские бренды (LEVRANA, ARAVIA PROFESSIONAL, BOTAVIKOS, ГУРМАНДИЗ). В Sephora представлены в основном международные премиальные и нишевые бренды (TOM FORD, Charlotte Tilbury, Oribe, Benefit Cosmetics и др.).

В Рив Гош бренды из Топ-10 имеют большее количество товаров (от 162 до 269 позиций). В Sephora ассортимент для Топ-10 брендов значительно меньше (от 75 до 91 позиций).

Можно сделать вывод, что Рив Гош ориентирован на более широкий ассортимент, включая массовые и профессиональные бренды, а также российские марки.
Sephora делает же предоставляет больший ассортимент люксовых брендов.

In [ ]:
offline_only_count = df_riv['only_in_stores'].sum()
offline_share = offline_only_count / len(df_riv) * 100
online_only_count = df_sephora['isOnlineOnly'].sum()
online_share = online_only_count / len(df_sephora) * 100

labels = ['Рив Гош', 'Sephora']
values_offline_only = [offline_share, 0]
values_online_only = [0, online_share]
values_mixed = [100-offline_share, 100-online_share]

fig = go.Figure(data=[
    go.Bar(name='Только офлайн', x=labels, y=values_offline_only, marker_color='#6a9955'),
    go.Bar(name='Только онлайн', x=labels, y=values_online_only, marker_color='#569cd6'),
    go.Bar(name='В обоих каналах', x=labels, y=values_mixed, marker_color='#d4d4d4')
])
fig.update_layout(
    title='Распределение товаров по каналам продаж',
    yaxis_title='Процент товаров',
    barmode='stack',
    width=800,
    height=500,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

fig.add_annotation(
    x='Рив Гош', y=offline_share/2,
    text=f"{offline_only_count} шт.<br>({offline_share:.1f}%)",
    showarrow=False,
    font=dict(color="white")
)

fig.add_annotation(
    x='Sephora', y=online_share/2,
    text=f"{online_only_count} шт.<br>({online_share:.1f}%)",
    showarrow=False,
    font=dict(color="white")
)

fig.show()

В обоих сетях подавляющее большинство товаров продается одновременно и в офлайне, и в онлайне. Это говорит о том, что обе компании придерживаются омниканальной стратегии, стараясь сделать ассортимент доступным как в розничных магазинах, так и в интернете.

У Рив Гош заметная доля товаров доступна только в офлайн-магазинах. Возможно, офлайн-магазины Рив Гош играют важную роль в стратегии компании, привлекая покупателей уникальным ассортиментом, который нельзя заказать онлайн. Также у некоторых товаров, например, витаминов и БАДов, могут быть ограничения на доставку.

В Sephora значительная доля товаров продается только онлайн. Это может говорить о развитой цифровой стратегии, когда сеть активно тестирует новые продукты и бренды через e-commerce. Возможно, Sephora сотрудничает с нишевыми брендами, которые пока не представлены в офлайн-магазинах.

Рив Гош ориентирован на офлайн, при этом поддерживая омниканальность.
Sephora делает акцент на уникальность товаров в онлайне, предлагая часть ассортимента исключительно там.

In [ ]:
fig = make_subplots(rows=1, cols=2,
                   subplot_titles=("Рейтинг товаров Рив Гош", "Рейтинг товаров Sephora"))
fig.add_trace(
    go.Histogram(
        x=df_riv['rating'].dropna(),
        nbinsx=50,
        xbins=dict(start=0, end=5, size=0.05),
        marker_color='salmon',
        marker_line_color='black',
        marker_line_width=1
    ),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(
        x=df_sephora['rating'].dropna(),
        nbinsx=50,
        xbins=dict(start=0, end=5, size=0.05),
        marker_color='orange',
        marker_line_color='black',
        marker_line_width=1
    ),
    row=1, col=2
)

fig.update_layout(
    height=400,
    width=1000,
    showlegend=False
)

fig.update_xaxes(title_text="Средняя оценка", range=[0, 5], row=1, col=1)
fig.update_xaxes(title_text="Средняя оценка", range=[0, 5], row=1, col=2)
fig.update_yaxes(title_text="Число товаров", row=1, col=1)
fig.update_yaxes(title_text="Число товаров", row=1, col=2)

fig.show()

Оба графика смещены вправо – это значит, что большинство товаров если и имеют оценки, то они высокие. Для Sephora пик около 4.5, для Рив Гош больше всего товаров с рейтингом 5. Однако необходимо отметить, что у Рив Гош очень много товаров вовсе без рейтинга.

Таким образом, можно сделать вывод, что вовлеченность покупателей Sephora выше, чем вовлеченность покупателей Рив Гош.



In [ ]:
fig = make_subplots(rows=1, cols=1,
                   subplot_titles=["Распределение рейтингов товаров"],
                   horizontal_spacing=0.1)

fig.add_trace(
    go.Box(
        y=df_riv['rating'].dropna(),
        name="Рив Гош",
        boxmean=True,
        marker_color='salmon',
        boxpoints='outliers',
        line=dict(width=2)
    )
)

fig.add_trace(
    go.Box(
        y=df_sephora['rating'].dropna(),
        name="Sephora",
        boxmean=True,
        marker_color='orange',
        boxpoints='outliers',
        line=dict(width=2)
    )
)

fig.update_layout(
    title_text="Сравнение распределения рейтингов в Рив Гош и Sephora",
    height=500,
    width=800,
    yaxis=dict(
        title="Рейтинг товаров",
        range=[0, 5.2],
        zeroline=True
    ),
    showlegend=True,
    boxmode='group'
)

fig.show()

Можем заметить, что товары Рив Гош имеют более высокие оценки, чем у Sephora. Это подтверждается высокой медианой и средним у Рив Гоша - 5 и 4.9 соответственно, а у Sephora данные показатели на уровне 4.4 по медиане и 4.3 по среднему. Также усы у Sephora длиннее, что говорит о более широком диапазоне значений между q1=3,36 и q3=4,6. В общем, можно отметить, что оценки товаров находятся на высоком уровне, что говорит об удовлетворенности клиентов и высоком качестве продукции. Однако, редкие случаи низких рейтингов (выбросы) требуют внимания и анализа для дальнейшего улучшения качества и минимизации негативного влияния на бизнес.

In [ ]:
# рассчитываем средний рейтинг и средний отнормированный рейтинг, цену
sephora_mean = df_sephora['rating'].mean()
sephora_std = df_sephora['rating'].std()
riv_mean = df_riv['rating'].mean()
riv_std = df_riv['rating'].std()

sephora_price_mean = df_sephora['price'].mean()
sephora_price_std = df_sephora['price'].std()
riv_price_mean = df_riv['price'].mean()
riv_price_std = df_riv['price'].std()

df_sephora_brands = df_sephora.groupby('brandName').agg({
    'rating': 'mean',
    'price': 'mean'
}).reset_index()
df_sephora_brands.columns = ['brand', 'avg_rating', 'avg_price']
df_sephora_brands['avg_rating_norm'] = (df_sephora_brands['avg_rating'] - sephora_mean) / sephora_std
df_sephora_brands['avg_price_norm'] = (df_sephora_brands['avg_price'] - sephora_price_mean) / sephora_price_std

df_riv_brands = df_riv.groupby('brand_name').agg({
    'rating': 'mean',
    'price': 'mean'
}).reset_index()
df_riv_brands.columns = ['brand', 'avg_rating', 'avg_price']
df_riv_brands['avg_rating_norm'] = (df_riv_brands['avg_rating'] - riv_mean) / riv_std
df_riv_brands['avg_price_norm'] = (df_riv_brands['avg_price'] - riv_price_mean) / riv_price_std

In [ ]:
df_sephora_brands.head()

,brand,avg_rating,avg_price,avg_rating_norm,avg_price_norm
0,ABBOTT,4.431773,56.909091,0.325722,0.005596
1,AERIN,4.109700,131.250000,-0.405036,1.155854
2,ALPYN,4.526180,48.733333,0.539925,-0.120905
3,ALTERNA Haircare,4.356491,45.181818,0.154913,-0.175857
4,Act+Acre,4.656597,52.838710,0.835830,-0.057384


In [ ]:
df_riv_brands.head()

,brand,avg_rating,avg_price,avg_rating_norm,avg_price_norm
0,100BON,4.763636,6019.545455,-0.668505,0.165715
1,27 87 Perfumes,4.970000,10259.090909,0.310495,0.643658
2,3LAB,NaN,54140.500000,NaN,5.590610
3,A-DERMA,NaN,1726.875000,NaN,-0.318218
4,ACCA KAPPA,5.000000,2434.333333,0.452817,-0.238463


In [ ]:
def normalize_brand_name(name):
    if pd.isna(name):
        return name
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name)
    name = name.strip()
    if name.endswith(' ru'):
        name = name[:-3]
    return name

In [ ]:
df_sephora_brands = df_sephora_brands[df_sephora_brands['avg_rating'].notnull()]
df_riv_brands = df_riv_brands[df_riv_brands['avg_rating'].notnull()]

df_sephora_brands['brand_normalized'] = df_sephora_brands['brand'].apply(normalize_brand_name)
df_riv_brands['brand_normalized'] = df_riv_brands['brand'].apply(normalize_brand_name)

df_sephora_agg = df_sephora_brands.groupby('brand_normalized').agg({
    'avg_rating': 'mean',
    'avg_rating_norm': 'mean',
    'avg_price': 'mean',
    'avg_price_norm': 'mean'
}).reset_index()

df_riv_agg = df_riv_brands.groupby('brand_normalized').agg({
    'avg_rating': 'mean',
    'avg_rating_norm': 'mean',
    'avg_price': 'mean',
    'avg_price_norm': 'mean'
}).reset_index()
common_brands = pd.merge(df_sephora_agg, df_riv_agg, on='brand_normalized', how='inner',
                         suffixes=('_sephora', '_riv'))

In [ ]:
common_brands.columns = [
    'brand',
    'sephora_rating',
    'sephora_rating_norm',
    'sephora_price',
    'sephora_price_norm',
    'riv_rating',
    'riv_rating_norm',
    'riv_price',
    'riv_price_norm'
]

In [ ]:
common_brands

,brand,sephora_rating,sephora_rating_norm,sephora_price,sephora_price_norm,riv_rating,riv_rating_norm,riv_price,riv_price_norm
0,beautyblender,4.415238,0.288207,23.076923,-0.517881,4.850000,-0.258791,2360.000000,-0.246843
1,bobbi brown,4.423831,0.307702,44.000000,-0.194143,4.967857,0.300329,5071.250000,0.058809
2,carolina herrera,4.380297,0.208928,83.457143,0.416367,4.933333,0.136547,17661.630435,1.478180
3,caudalie,4.380343,0.209033,48.698113,-0.121450,4.856364,-0.228602,4283.370787,-0.030012
4,clarins,4.384712,0.218944,76.264706,0.305080,4.914516,0.047277,4886.142322,0.037941
5,clinique,4.111670,-0.400566,40.844595,-0.242966,4.768571,-0.645093,4739.715909,0.021434
6,dr barbara sturm,4.160450,-0.289889,182.083333,1.942385,5.000000,0.452817,20701.282051,1.820854
7,givenchy,4.317409,0.066239,38.913043,-0.272852,4.913333,0.041665,NaN,NaN
8,gucci,4.257045,-0.070723,87.229730,0.474740,5.000000,0.452817,13476.500000,1.006371
9,guerlain,4.328400,0.091177,116.935484,0.934369,4.950000,0.215614,NaN,NaN


In [ ]:
common_brands['brand'] = common_brands['brand'].str.title()

sephora_avg = common_brands['sephora_rating'].mean()
riv_avg = common_brands['riv_rating'].mean()

sephora_norm_avg = common_brands['sephora_rating_norm'].mean()
riv_norm_avg = common_brands['riv_rating_norm'].mean()

fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['sephora_rating'],
    name='Sephora',
    marker_color='orange',
    text=common_brands['sephora_rating'].round(2),
    textposition='auto'
))
fig1.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['riv_rating'],
    name='Рив Гош',
    marker_color='salmon',
    text=common_brands['riv_rating'].round(2),
    textposition='auto'
))

fig1.add_shape(
    type="line",
    x0=-0.5,
    y0=sephora_avg,
    x1=len(common_brands['brand']),
    y1=sephora_avg,
    line=dict(
        color="orange",
        width=2,
        dash="dash",
    )
)
fig1.add_shape(
    type="line",
    x0=-0.5,
    y0=riv_avg,
    x1=len(common_brands['brand']),
    y1=riv_avg,
    line=dict(
        color="salmon",
        width=2,
        dash="dash",
    )
)

fig1.update_layout(
    title='Сравнение средних рейтингов брендов в Sephora и Рив Гош',
    xaxis_title='Бренд',
    yaxis_title='Средний рейтинг',
    yaxis=dict(range=[0, 5]),
    barmode='group',
    height=600,
    width=1000,
    bargap=0.15,
    bargroupgap=0.1,
    legend=dict(
        x=0.01,
        y=0.01,
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)
fig1.update_xaxes(tickangle=45)
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['sephora_rating_norm'],
    name='Sephora',
    marker_color='orange',
    text=common_brands['sephora_rating_norm'].round(2),
    textposition='auto'
))
fig2.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['riv_rating_norm'],
    name='Рив Гош',
    marker_color='salmon',
    text=common_brands['riv_rating_norm'].round(2),
    textposition='auto'
))

y_min = min(common_brands['sephora_rating_norm'].min(), common_brands['riv_rating_norm'].min())
y_max = max(common_brands['sephora_rating_norm'].max(), common_brands['riv_rating_norm'].max())
y_range = [y_min - 0.3, y_max + 0.3]

fig2.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(common_brands['brand']),
    y1=0,
    line=dict(
        color="gray",
        width=1,
        dash="dash",
    )
)
fig2.add_shape(
    type="line",
    x0=-0.5,
    y0=sephora_norm_avg,
    x1=len(common_brands['brand']),
    y1=sephora_norm_avg,
    line=dict(
        color="orange",
        width=2,
        dash="dash",
    )
)
fig2.add_shape(
    type="line",
    x0=-0.5,
    y0=riv_norm_avg,
    x1=len(common_brands['brand']),
    y1=riv_norm_avg,
    line=dict(
        color="salmon",
        width=2,
        dash="dash",
    )
)

fig2.update_layout(
    title='Сравнение нормализованных рейтингов брендов в Sephora и Рив Гош',
    xaxis_title='Бренд',
    yaxis_title='Нормализованный рейтинг',
    yaxis=dict(range=y_range, zeroline=False),
    barmode='group',
    height=600,
    width=1000,
    bargap=0.15,
    bargroupgap=0.1,
    legend=dict(
        x=0.01,
        y=0.01,
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)
fig2.update_xaxes(tickangle=45)
fig2.show()

Гипотеза о том, что в Рив Гош бренды лучше, провалилась, т.к. даже на совпадающих брендах у Рив Гош рейтинг выше, чем у Sephora.

При этом рейтинг совпадающих брендов в Рив Гош выше, чем в Sephora, относительно других брендов магазина.

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Распределение цен в Sephora', 'Распределение цен в Рив Гош'))
sephora_filtered = df_sephora[df_sephora['price'] <= 500]
riv_filtered = df_riv[df_riv['price'] <= 50000]

fig.add_trace(
    go.Box(
        y=sephora_filtered['price'],
        name='Sephora',
        marker_color='orange',
        boxmean=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Box(
        y=riv_filtered['price'],
        name='Рив Гош',
        marker_color='salmon',
        boxmean=True
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='Ящики с усами для распределения цен Sephora и Рив Гош',
    height=600,
    width=1000,
    showlegend=False
)

fig.update_yaxes(title_text='Цена, $', row=1, col=1)
fig.update_yaxes(title_text='Цена, ₽', row=1, col=2)

fig.show()

На данном графике можно рассмотреть ящики с усами для стоимости товаров у двух компаний. Можно заметить, что цены на косметику в сефоре на порядок выше чем в Рив Гош: по среднему 55 дол и 4000 руб., по медиане 36 дол. и 1500 руб.. Также это наглядно отображено на графике по расположению самого ящика, тк q1 у графиков компаний составляет 26 дол. и 620 руб. соответственно, а q3 на уровне 62 дол. и 4150 руб.. Также присутствуют выбросы вплоть до 500 дол. (50 000 руб.). Верхний ус имеет не такую большую разницу 116$ и 9400 руб.. Если считать, что 1 дол. = 100 руб. то можно наглядно заметить, что в сефоре более широкий диапазон цен, что говорит о большем наборе товаров разных ценовых категорий. А смещенная сниз медиана говорит о том, что во второй половине значенйи более широкий разброс цен.

In [ ]:
bins = [-0.1, 0.9, 10, 50, 200, np.inf]
labels = ["0", "1", "2-10", "11-50", "51-200+"]

df_riv['reviews_bin'] = pd.cut(df_riv['reviews_count'], bins=bins, labels=labels)
df_sephora['reviews_bin'] = pd.cut(df_sephora['reviews'], bins=bins, labels=labels)

rg_counts = df_riv['reviews_bin'].value_counts().reindex(labels)
sp_counts = df_sephora['reviews_bin'].value_counts().reindex(labels)

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Рив Гош: распределение по кол-ву отзывов",
                                    "Sephora: распределение по кол-ву отзывов"))
fig.add_trace(
    go.Bar(
        x=rg_counts.index,
        y=rg_counts.values,
        marker_color='lightblue',
        marker_line_color='black',
        marker_line_width=1,
        text=rg_counts.values,
        textposition='auto'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=sp_counts.index,
        y=sp_counts.values,
        marker_color='plum',
        marker_line_color='black',
        marker_line_width=1,
        text=sp_counts.values,
        textposition='auto'
    ),
    row=1, col=2
)

fig.update_layout(
    height=500,
    width=1000,
    showlegend=False,
    title_text="Распределение товаров по количеству отзывов"
)
fig.update_xaxes(title_text="Число отзывов на товар", row=1, col=1)
fig.update_xaxes(title_text="Число отзывов на товар", row=1, col=2)
fig.update_yaxes(title_text="Количество товаров", row=1, col=1)
fig.update_yaxes(title_text="Количество товаров", row=1, col=2)

fig.show()

Пользователи Sephora явно гораздо более вовлечены, чем пользователи Рив Гош. Возможно, Sephora каким-либо образом поощряет пользователей за публикацию отзывов на товары (бонусные баллы и др.).

С точки зрения бизнеса, обратная связь о продукции весьма важный аспект, так как ориентируясь на него компании могут оценить удовлетворенность аудитории товарами и его качеством, чтобы впоследствии грамотно корректировать ассортимент и повышать лояльность клиентов.

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Распределение цен в Sephora (до $500)', 'Распределение цен в Рив Гош (до ₽50,000)'))

sephora_filtered = df_sephora[df_sephora['price'] <= 500]
riv_filtered = df_riv[df_riv['price'] <= 50000]

sephora_mean = sephora_filtered['price'].mean()
sephora_median = sephora_filtered['price'].median()

riv_mean = riv_filtered['price'].mean()
riv_median = riv_filtered['price'].median()

fig.add_trace(
    go.Histogram(
        x=sephora_filtered['price'],
        nbinsx=100,
        marker_color='orange',
        opacity=0.7,
        name='Sephora'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(
        x=riv_filtered['price'],
        nbinsx=100,
        marker_color='salmon',
        opacity=0.7,
        name='Рив Гош'
    ),
    row=1, col=2
)

fig.add_shape(
    type="line",
    x0=sephora_mean, y0=0,
    x1=sephora_mean, y1=sephora_filtered['price'].value_counts().max() * 1.2,
    line=dict(color="red", width=2, dash="solid"),
    row=1, col=1
)

fig.add_shape(
    type="line",
    x0=sephora_median, y0=0,
    x1=sephora_median, y1=sephora_filtered['price'].value_counts().max() * 1.2,
    line=dict(color="blue", width=2, dash="dash"),
    row=1, col=1
)

fig.add_shape(
    type="line",
    x0=riv_mean, y0=0,
    x1=riv_mean, y1=riv_filtered['price'].value_counts().max() * 1.2,
    line=dict(color="red", width=2, dash="solid"),
    row=1, col=2
)

fig.add_shape(
    type="line",
    x0=riv_median, y0=0,
    x1=riv_median, y1=riv_filtered['price'].value_counts().max() * 1.2,
    line=dict(color="blue", width=2, dash="dash"),
    row=1, col=2
)

fig.add_annotation(
    x=sephora_mean, y=sephora_filtered['price'].value_counts().max() * 1.1,
    text=f"Mean: ${sephora_mean:.2f}",
    showarrow=True,
    arrowhead=1,
    row=1, col=1
)

fig.add_annotation(
    x=sephora_median, y=sephora_filtered['price'].value_counts().max() * 0.9,
    text=f"Median: ${sephora_median:.2f}",
    showarrow=True,
    arrowhead=1,
    row=1, col=1
)

fig.add_annotation(
    x=riv_mean, y=riv_filtered['price'].value_counts().max() * 3,
    text=f"Mean: ₽{riv_mean:.2f}",
    showarrow=True,
    arrowhead=1,
    row=1, col=2
)

fig.add_annotation(
    x=riv_median, y=riv_filtered['price'].value_counts().max() * 0.9,
    text=f"Median: ₽{riv_median:.2f}",
    showarrow=True,
    arrowhead=1,
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='red', width=2),
        name='Среднее значение',
        showlegend=True
    )
)

fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='blue', width=2, dash='dash'),
        name='Медиана',
        showlegend=True
    )
)

fig.update_layout(
    title_text='Сравнение распределения цен между Sephora и Рив Гош',
    height=500,
    width=1000,
    bargap=0.05,
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor='top',
        y=-0.15,
        xanchor='center',
        x=0.5
    )
)

fig.update_xaxes(title_text='Цена, $', range=[0, 500], row=1, col=1)
fig.update_xaxes(title_text='Цена, ₽', range=[0, 50000], row=1, col=2)
fig.update_yaxes(title_text='Количество товаров', row=1, col=1)
fig.update_yaxes(title_text='Количество товаров', row=1, col=2)

fig.show()

Оба графика распределения смещены влево - в основном цены не очень высокие. Позднее приведено сравнение цен уже в одной валюте.

In [ ]:
common_brands['brand'] = common_brands['brand'].str.title()
conversion_rate = 90
common_brands['sephora_price_rub'] = common_brands['sephora_price'] * conversion_rate

sephora_price_rub_avg = common_brands['sephora_price_rub'].mean()
riv_price_avg = common_brands['riv_price'].mean()
sephora_price_norm_avg = common_brands['sephora_price_norm'].mean()
riv_price_norm_avg = common_brands['riv_price_norm'].mean()

fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['sephora_price_rub'],
    name='Sephora (₽)',
    marker_color='orange',
    text=common_brands['sephora_price_rub'].round(0),
    textposition='auto'
))
fig1.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['riv_price'],
    name='Рив Гош (₽)',
    marker_color='salmon',
    text=common_brands['riv_price'].round(0),
    textposition='auto'
))

fig1.add_shape(
    type="line",
    x0=-0.5,
    y0=sephora_price_rub_avg,
    x1=len(common_brands['brand']),
    y1=sephora_price_rub_avg,
    line=dict(
        color="orange",
        width=2,
        dash="dash",
    )
)
fig1.add_shape(
    type="line",
    x0=-0.5,
    y0=riv_price_avg,
    x1=len(common_brands['brand']),
    y1=riv_price_avg,
    line=dict(
        color="salmon",
        width=2,
        dash="dash",
    )
)

fig1.update_layout(
    title='Сравнение средних цен брендов в Sephora и Рив Гош (₽)',
    xaxis_title='Бренд',
    yaxis_title='Средняя цена (₽)',
    barmode='group',
    height=600,
    width=1000,
    bargap=0.15,
    bargroupgap=0.1,
    legend=dict(
        x=0.01,
        y=0.01,
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)
fig1.update_xaxes(tickangle=45)
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['sephora_price_norm'],
    name='Sephora',
    marker_color='orange',
    text=common_brands['sephora_price_norm'].round(2),
    textposition='auto'
))
fig2.add_trace(go.Bar(
    x=common_brands['brand'],
    y=common_brands['riv_price_norm'],
    name='Рив Гош',
    marker_color='salmon',
    text=common_brands['riv_price_norm'].round(2),
    textposition='auto'
))

y_min = min(common_brands['sephora_price_norm'].min(), common_brands['riv_price_norm'].min())
y_max = max(common_brands['sephora_price_norm'].max(), common_brands['riv_price_norm'].max())
y_range = [y_min - 0.3, y_max + 0.3]

fig2.add_shape(
    type="line",
    x0=-0.5,
    y0=0,
    x1=len(common_brands['brand']),
    y1=0,
    line=dict(
        color="gray",
        width=1,
        dash="dash",
    )
)
fig2.add_shape(
    type="line",
    x0=-0.5,
    y0=sephora_price_norm_avg,
    x1=len(common_brands['brand']),
    y1=sephora_price_norm_avg,
    line=dict(
        color="orange",
        width=2,
        dash="dash",
    )
)
fig2.add_shape(
    type="line",
    x0=-0.5,
    y0=riv_price_norm_avg,
    x1=len(common_brands['brand']),
    y1=riv_price_norm_avg,
    line=dict(
        color="salmon",
        width=2,
        dash="dash",
    )
)

fig2.update_layout(
    title='Сравнение нормализованных цен брендов в Sephora и Рив Гош',
    xaxis_title='Бренд',
    yaxis_title='Нормализованная цена',
    yaxis=dict(range=y_range, zeroline=False),
    barmode='group',
    height=600,
    width=1000,
    bargap=0.15,
    bargroupgap=0.1,
    legend=dict(
        x=0.01,
        y=0.01,
        bgcolor='rgba(255, 255, 255, 0.5)'
    )
)
fig2.update_xaxes(tickangle=45)
fig2.show()

Сравнение средних цен продукции брендов, которые представлены в обеих сетях, показывает, что практически для всех брендов в Рив Гош цена выше, чем в Sephora (цены в рублях, курс = 90). Различия в цепочке поставок, включая импортные пошлины, логистические расходы и валютные курсы, могут влиять на конечную цену товара в разных сетях. Также не исключаются различия в ценообразовании.

In [ ]:
cat_counts = df_riv['category'].value_counts().head(10)

fig = make_subplots(rows=2, cols=1,
                   subplot_titles=("Распределение товаров Рив Гош по категориям (топ-10)",
                                  "Средняя цена товаров по категориям"),
                   vertical_spacing=0.3,
                   row_heights=[0.5, 0.5])

fig.add_trace(go.Bar(
    x=cat_counts.index,
    y=cat_counts.values,
    marker_color='pink',
    text=cat_counts.values,
    textposition='auto',
    name='Количество товаров'
), row=1, col=1)

avg_prices = []
for category in cat_counts.index:
    avg_price = df_riv[df_riv['category'] == category]['price'].mean()
    avg_prices.append(avg_price)

fig.add_trace(go.Bar(
    x=cat_counts.index,
    y=avg_prices,
    marker_color='salmon',
    text=[f"{price:.0f} ₽" for price in avg_prices],
    textposition='auto',
    name='Средняя цена'
), row=2, col=1)

fig.update_layout(
    height=800,
    width=800,
    showlegend=False
)

fig.update_xaxes(tickangle=45, row=1, col=1)
fig.update_xaxes(tickangle=45, row=2, col=1)
fig.update_yaxes(title_text="Количество товаров", row=1, col=1)
fig.update_yaxes(title_text="Средняя цена (₽)", row=2, col=1)

fig.show()

Распределению товаров по категориям в Рив Гош показывает, что наибольшее количество позиций присутствует в категории "Парфюмерия". Дело в том, что изначально в Рив Гош была представлена только парфюмерия, а на данный момент бренд успешно диверсифицировал свой ассортимент.

Кроме того, данное распределение позволяет выявить "зоны роста", то есть те категории, в которых можно увеличить количество предлагаемых позиций (например, уход за руками и ногтями и солнцезащита и загар).

Если говорить о средних ценах по категориям, то наибольшие у парфюмерии, аксессуаров и различных наборов.

In [ ]:
fig = make_subplots(rows=2, cols=2,
                   subplot_titles=("Корреляция Пирсона - Sephora", "Корреляция Пирсона - Рив Гош",
                                  "Корреляция Спирмана - Sephora", "Корреляция Спирмана - Рив Гош"),
                   vertical_spacing=0.15,
                   horizontal_spacing=0.15)

sephora_data = df_sephora[['price', 'rating']].dropna()
sephora_corr_pearson = sephora_data.corr(method='pearson')
riv_data = df_riv[['price', 'rating']].dropna()
riv_corr_pearson = riv_data.corr(method='pearson')
sephora_corr_spearman = sephora_data.corr(method='spearman')
riv_corr_spearman = riv_data.corr(method='spearman')
fig.add_trace(
    go.Heatmap(
        z=sephora_corr_pearson.values,
        x=['Цена', 'Рейтинг'],
        y=['Цена', 'Рейтинг'],
        colorscale='Reds',
        zmin=-1, zmax=1,
        text=np.round(sephora_corr_pearson.values, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=riv_corr_pearson.values,
        x=['Цена', 'Рейтинг'],
        y=['Цена', 'Рейтинг'],
        colorscale='Reds',
        zmin=-1, zmax=1,
        text=np.round(riv_corr_pearson.values, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
    ),
    row=1, col=2
)

fig.add_trace(
    go.Heatmap(
        z=sephora_corr_spearman.values,
        x=['Цена', 'Рейтинг'],
        y=['Цена', 'Рейтинг'],
        colorscale='Reds',
        zmin=-1, zmax=1,
        text=np.round(sephora_corr_spearman.values, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
    ),
    row=2, col=1
)

fig.add_trace(
    go.Heatmap(
        z=riv_corr_spearman.values,
        x=['Цена', 'Рейтинг'],
        y=['Цена', 'Рейтинг'],
        colorscale='Reds',
        zmin=-1, zmax=1,
        text=np.round(riv_corr_spearman.values, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
    ),
    row=2, col=2
)

fig.update_layout(
    height=800,
    width=900,
    title_text="Корреляция между ценой и рейтингом",
)

fig.show()

Можно заметить, что в обоих типах корреляции и в обоих магазинах очень низкая корреляция между рейтингом и ценой (близка к 0). Практически незначима, то есть рейтинг зависит от комплекса факторов, в который цена может входить частично.

In [ ]:
corr_columns = ['sephora_rating', 'riv_rating', 'sephora_price', 'riv_price']

pearson_corr = common_brands[corr_columns].corr()
spearman_corr = common_brands[corr_columns].corr(method='spearman')

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Корреляция Пирсона", "Корреляция Спирмана"),
    horizontal_spacing=0.15
)

fig.add_trace(
    go.Heatmap(
        z=pearson_corr.values,
        x=['Рейтинг Sephora', 'Рейтинг Рив Гош', 'Цена Sephora', 'Цена Рив Гош'],
        y=['Рейтинг Sephora', 'Рейтинг Рив Гош', 'Цена Sephora', 'Цена Рив Гош'],
        colorscale='RdBu_r',
        zmin=-1, zmax=1,
        text=np.round(pearson_corr.values, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
    ),
    row=1, col=1
)
fig.add_trace(
    go.Heatmap(
        z=spearman_corr.values,
        x=['Рейтинг Sephora', 'Рейтинг Рив Гош', 'Цена Sephora', 'Цена Рив Гош'],
        y=['Рейтинг Sephora', 'Рейтинг Рив Гош', 'Цена Sephora', 'Цена Рив Гош'],
        colorscale='RdBu_r',
        zmin=-1, zmax=1,
        text=np.round(spearman_corr.values, 2),
        texttemplate='%{text}',
        textfont={"size": 14},
    ),
    row=1, col=2
)

fig.update_layout(
    title="Корреляция между ценами и рейтингами для общих брендов Sephora и Рив Гош",
    height=600,
    width=1200,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_title="",
    yaxis_title="",
)

fig.show()

В обоих магазинах наблюдается слабая или умеренная отрицательная корреляция между ценой и рейтингом. Это может указывать на то, что более дорогие товары не обязательно получают более высокие рейтинги.
Также наблюдается сильная положительная корреляция между ценами в Рив Гош и Sephora (0.8 по Пирсону и 0.89 по Спирману), которая указывает на то, что цены на общие бренды в обоих магазинах имеют схожую тенденцию. То же самое отлеживается в корреляции рейтингов магазинов - 0.47 по Пирсону и 0.43 по Спирману соответственно.